In [1]:
import numpy as np

In [ ]:
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

# restore np.load for future normal usage
np.load = np_load_old

In [3]:
X_train, y_train, X_test, y_test, X_dev, y_dev = np.load('all_data.npy', allow_pickle = True)

In [8]:
X_train.shape

(767,)